In [90]:
import re
import pickle
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from string import punctuation
from keras.preprocessing.text import Tokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
#import math
import itertools

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [91]:
file = open("/content/brown.txt","r")
data = file.read()
data = data.lower()
#data = remove_stopwords(data)
#print(data)
all_text = ''.join([c for c in data if c not in punctuation])
#print(len(all_text))
text_split = all_text.split('\n')


In [92]:
def create_seq(text, seq_len = 1):
    
    sequences = []

    # if the number of tokens in 'text' is greater than 5
    if len(text.split()) > seq_len:
      for i in range(seq_len, len(text.split())):
        # select sequence of tokens
        seq = text.split()[i-seq_len:i+1]
        # add to the list
        sequences.append(" ".join(seq))
        

      return sequences

    # if the number of tokens in 'text' is less than or equal to 5
    else:
      
      return [text]

In [93]:
words_list = [create_seq(i) for i in text_split ]

# merge list-of-lists into a single list
#words_list = sum(words_list, [])
count=len(words_list)
# count of sequences
random.shuffle(words_list)
train = words_list[:math.ceil(count*0.7)]
validation = words_list[math.ceil(count*0.7):math.ceil(count*0.7)+math.ceil(count*0.2)]
test = words_list[math.ceil(count*0.7)+math.ceil(count*0.2):]
print(train[0])
#print(seqs[0])
#print(seqs)

['but you', 'you dont', 'dont have', 'have to', 'to worship', 'worship in', 'in the', 'the traditional', 'traditional way', 'way you', 'you can', 'can communicate']


In [95]:
train_data = list(itertools.chain.from_iterable(train))
x_train = []
y_train = []

for s in train_data:
  x_train.append(" ".join(s.split()[:-1]))
  y_train.append(" ".join(s.split()[1:]))

In [97]:
validation_data = list(itertools.chain.from_iterable(validation))
x_val = []
y_val = []

for s in validation_data:
  x_val.append(" ".join(s.split()[:-1]))
  y_val.append(" ".join(s.split()[1:]))

In [99]:
#print(x)
print(x_train[:100])

['but', 'you', 'dont', 'have', 'to', 'worship', 'in', 'the', 'traditional', 'way', 'you', 'can', 'at', 'the', 'same', 'time', 'the', 'usefulness', 'of', 'this', 'study', 'and', 'of', 'like', 'a', 'great', 'balloon', 'a', 'hundred', 'yards', 'across', 'a', 'balloon', 'big', 'davidson', 'throwing', 'only', '17', 'passes', 'we', 'always', 'like', 'to', 'on', 'pages', '8', 'to', '14', 'land', 'three', 'quarters', 'to', '1', 'acre', 'of', 'good', 'the', 'senate', 'tonight', 'confirmed', 'robert', 'c', 'weaver', 'as', 'the', 'because', 'of', 'the', 'lack', 'of', 'juvenile', 'accommodations', 'at', 'the', 'anne', 'him', 'she', 'was', 'wise', 'enough', 'to', 'realize', 'a', 'man', 'could', 'be', 'good', 'company', 'what', 'mae', 'would', 'want', 'him', 'to', 'do', 'though', 'he', 'was', 'sure', 'of']


In [100]:
int2token = {}
cnt = 0

for w in set(" ".join(text_split).split()):
  int2token[cnt] = w
  cnt+= 1

# create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

In [101]:
vocab_size = len(int2token)
vocab_size

48171

In [102]:
def get_integer_seq(seq):
  return [token2int[w] for w in seq.split()]
#x_train_int=np.array([])
# convert text sequences to integer sequences
x_int = [get_integer_seq(i) for i in x_train]
x_int = list(filter(None, x_int))
y_int = [get_integer_seq(i) for i in y_train]
y_int = list(filter(None, y_int))
print(x_int[:50])
# convert lists to numpy arrays
x_train_int = np.asarray(x_int)
y_train_int = np.asarray(y_int)

print(y_train_int)

[[25574], [10179], [34823], [16173], [40446], [1215], [41476], [4337], [23218], [29641], [10179], [7762], [37104], [4337], [107], [47185], [4337], [7346], [36764], [26876], [19345], [45870], [36764], [4101], [28880], [15923], [22623], [28880], [46405], [30877], [3043], [28880], [22623], [37364], [31989], [30901], [40884], [23723], [14823], [47804], [10211], [4101], [40446], [42904], [27152], [46383], [40446], [33035], [42783], [36218]]
[[10179]
 [34823]
 [16173]
 ...
 [47716]
 [33637]
 [43598]]


In [103]:
x_int = [get_integer_seq(i) for i in x_val]
y_int = [get_integer_seq(i) for i in y_val]
x_int = list(filter(None, x_int))
y_int = list(filter(None, y_int))
# convert lists to numpy arrays
x_val_int = np.array(x_int)
y_val_int = np.array(y_int)
print(x_val_int)

[[43613]
 [28880]
 [36764]
 ...
 [25892]
 [ 6993]
 [41422]]


In [104]:
def get_batches(arr_x, arr_y, batch_size):
         
    prv = 0
    for n in range(batch_size, arr_x.shape[0], batch_size):
      x = arr_x[prv:n]
      y = arr_y[prv:n]
      prv = n
      yield x, y

In [105]:
class WordLSTM(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=1, drop_prob=0.3, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding(vocab_size, 200)
        self.lstm = nn.LSTM(200, n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        embedded = self.emb_layer(x)     
        lstm_output, hidden = self.lstm(embedded, hidden)
        out = self.dropout(lstm_output)
        out = out.reshape(-1, self.n_hidden) 

        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        # if GPU is available
        if (torch.cuda.is_available()):
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        # if GPU is not available
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [106]:
net = WordLSTM()
print(net)

WordLSTM(
  (emb_layer): Embedding(48171, 200)
  (lstm): LSTM(200, 256, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=48171, bias=True)
)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [107]:
def training(net, epochs=10, batch_size=32, lr=0.001, clip=1, print_every=32):
    
    # optimizer
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    
    # loss
    criterion = nn.CrossEntropyLoss()
    
    # push model to GPU
    net.cuda()
    
    counter = 0

    net.train()

    for e in range(epochs):

        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(x_train_int, y_train_int, batch_size):
            counter+= 1
            
            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            # push tensors to GPU
            inputs, targets = inputs.cuda(), targets.cuda()

            # detach hidden states
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1))

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            # update weigths
            opt.step()            
            
            if counter % print_every == 0:
            
              print("Epoch: {}/{}...".format(e+1, epochs),
                    "Step: {}...".format(counter),"loss-->"+str(loss))

In [108]:
training(net, batch_size = 256, epochs=5, print_every=256)

Epoch: 1/5... Step: 256... loss-->tensor(7.0977, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 1/5... Step: 512... loss-->tensor(7.4440, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 1/5... Step: 768... loss-->tensor(7.0116, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 1/5... Step: 1024... loss-->tensor(6.9082, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 1/5... Step: 1280... loss-->tensor(6.9837, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 1/5... Step: 1536... loss-->tensor(7.1317, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 1/5... Step: 1792... loss-->tensor(6.7556, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 1/5... Step: 2048... loss-->tensor(6.5793, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 1/5... Step: 2304... loss-->tensor(6.9576, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 2/5... Step: 2560... loss-->tensor(6.5458, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch: 2/5... Step: 2816... loss-->tensor(6.6125, device='cuda:

In [109]:
def predict(net, tkn, h=None):
         
  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to GPU
  inputs = inputs.cuda()

  # detach hidden state from history
  h = tuple([each.data for each in h])

  out, h = net(inputs, h)
  
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  return np.amax(p)


def sample(net, size, prime='it is'):

    net.cuda()
    
    net.eval()

    h = net.init_hidden(1)

    toks = prime.split()

    for t in prime.split():
      prob = predict(net, t, h)
    
    return prob

In [110]:
tr=[]
for li in train:
  arr=[]
  for w in li:
    arr.append(w.split())
  flat_list = list(itertools.chain(*arr))
  res = [i for n, i in enumerate(flat_list) if i not in flat_list[:n]]
  tr.append(res)
print(tr[:5])

[['but', 'you', 'dont', 'have', 'to', 'worship', 'in', 'the', 'traditional', 'way', 'can', 'communicate'], ['at', 'the', 'same', 'time', 'usefulness', 'of', 'this', 'study', 'and', 'configuration'], ['like', 'a', 'great', 'balloon', 'hundred', 'yards', 'across', 'big', 'enough'], ['davidson', 'throwing', 'only', '17', 'passes', 'we', 'always', 'like', 'to', 'keep'], ['on', 'pages', '8', 'to', '14', 'land', 'three', 'quarters', '1', 'acre', 'of', 'good']]


In [128]:
final=[]
tr = list(filter(None, tr))
for lst in tr:
  result=[]
  for wrd in lst:
    result.append(sample(net, 1, prime = wrd))
  final.append(result)
#print(final)
print(final[:5])

[[0.08845882, 0.06342761, 0.086047046, 0.121885024, 0.11571468, 0.052822545, 0.23338042, 0.007917744, 0.0141615225, 0.074246086, 0.25857925, 0.06301818], [0.31975278, 0.007917744, 0.13407178, 0.052594524, 0.25035557, 0.19453984, 0.03141394, 0.1625606, 0.048843816, 0.16115439], [0.22603874, 0.01167673, 0.059540827, 0.026936613, 0.12185813, 0.02602833, 0.5715366, 0.024259165, 0.24075618], [0.0028197968, 0.012196671, 0.051917166, 0.046450377, 0.047641724, 0.06675202, 0.035733666, 0.22603874, 0.11571468, 0.06112344], [0.25111258, 0.0245902, 0.04294713, 0.11571468, 0.018339694, 0.04767761, 0.10392941, 0.06990113, 0.051159833, 0.10821416, 0.19453984, 0.03773212]]


In [133]:
perplexity_train=[]
sum=0
cnt=0
c=0
for l in final:
  probability = np.prod(l)
  if probability == 0:
    probability = 0.075
  res = probability ** (-1/len(l))
  perplexity_train.append(res)
  cnt+=1
print(perplexity_train)
avg_train = math.fsum(perplexity_train)/cnt
print("average perplexity of train dataset: "+str(avg_train))

[14.519938791745107, 11.108491926113837, 14.120099798043753, 24.109777904745442, 14.965758107215104, 43.334949381398204, 10.378564780756466, 14.864225120615915, 10.47259141007275, 19.053798962965974, 21.676487320874436, 17.998923082169394, 24.00017529455654, 19.040737940395946, 21.277955625115116, 16.666117953006847, 19.247186188476054, 11.698936793311148, 21.425244618720406, 19.750647603051213, 25.939132660515035, 6.858342651628364, 23.268027527805735, 15.400920125295846, 23.192955750216093, 15.48234656532916, 18.805106233850026, 19.262232005400815, 24.859512916923848, 24.701761331543203, 27.044210242565917, 13.813178338714238, 14.939427157950615, 15.80645271229886, 21.074057539368958, 22.027864972507672, 14.377307045656398, 30.724267536028716, 27.590970994108286, 22.94249095889987, 12.689929506098316, 19.175535934332174, 19.903790281223426, 25.68168869941605, 17.357425191763863, 13.907696712606132, 22.26041476013366, 17.87428764407998, 16.005103583280682, 22.137629816432796, 16.95488

In [134]:
f1 = open("/content/train_file_neural model.txt","w")
j = 0
for ls in tr:
  sentence = " ".join(ls)
  f1.write(sentence + "\t" + str(perplexity_train[j]) + "\n")
  j += 1
f1.write("\n")
f1.write("average perplexity of test dataset: "+str(avg_train))

54

In [115]:


val=[]
for li in validation:
  arr=[]
  for w in li:
    arr.append(w.split())
  flat_list = list(itertools.chain(*arr))
  res = [i for n, i in enumerate(flat_list) if i not in flat_list[:n]]
  val.append(res)
print(val[:5])

[['1', 'a', 'of', 'the', 'agreement', 'government'], ['mrs', 'ten', 'eyke', 'did', 'mental', 'health', 'trenchard', 'worked', 'for'], ['it', 'alexandre', 'livshitz', 'repeated', 'a', 'fantastic'], ['up', 'with', 'him', 'vocally', 'and', 'his', 'allegro', 'movements', 'around', 'the', 'luncheon'], ['would', 'have', 'preferred', 'death', 'and', 'bankruptcy', 'to', 'having', 'his', 'son', 'stay', 'with']]


In [114]:


final=[]
val = list(filter(None, val))
for lst in val:
  result=[]
  for wrd in lst:
    result.append(sample(net, 1, prime = wrd))
  final.append(result)
#print(final)
print(final[:5])


[[0.051159833, 0.01167673, 0.19453984, 0.007917744, 0.027300362, 0.057276256], [0.049148183, 0.3525562, 0.0085505685, 0.42808968, 0.01870345, 0.05211461, 0.024212528, 0.08388315, 0.13848545], [0.12576813, 0.0046486, 0.07680218, 0.011757502, 0.01167673, 0.0038324904], [0.12499176, 0.1361429, 0.07236932, 0.036099862, 0.048843816, 0.019672146, 0.0073842555, 0.040290866, 0.19938439, 0.007917744, 0.04322112], [0.21651098, 0.121885024, 0.048322674, 0.046141632, 0.048843816, 0.01583627, 0.11571468, 0.059891045, 0.019672146, 0.058911584, 0.035064995, 0.1361429]]


In [116]:

perplexity=[]
sum=0
cnt=0
c=0
for l in final:
  probability = np.prod(l)
  if probability == 0:
    probability = 0.075
  res = probability ** (-1/len(l))
  perplexity.append(res)
  cnt+=1
print(perplexity)
avg = math.fsum(perplexity)/cnt
print("average perplexity of validation dataset: "+str(avg))
 

[29.762273249980048, 15.686498638459954, 59.03406939744251, 23.402575932433326, 16.97654517936383, 28.308966831236162, 16.298799523379873, 13.06878397946662, 19.324944947058142, 22.576989103241406, 17.78195269145527, 14.104255824520088, 31.610957697877602, 19.005230335773355, 20.817085140487382, 23.303711952360175, 17.70093939079288, 10.392467875719623, 39.22174603224992, 16.394004107031463, 34.95304378433389, 14.444654933537919, 15.771670016006224, 10.948746089570255, 19.82210134037652, 14.6872631412403, 15.060283610973826, 7.951140239587989, 17.591151394520747, 17.97085239298923, 22.364359147065368, 23.517233953964862, 28.51094242732043, 18.275035539428732, 32.727526717851205, 28.266744315236465, 15.740762242008818, 25.663561023409112, 24.867513863974185, 39.97434433409126, 27.37640723181521, 13.518223981172278, 25.2800758229154, 17.259940385102123, 15.256766816601663, 23.108926333995363, 22.54016697828768, 22.354500684386416, 17.74658908568048, 23.09837653536134, 28.167655413735513,

In [117]:
te=[]
for li in test:
  arr=[]
  for w in li:
    arr.append(w.split())
  flat_list = list(itertools.chain(*arr))
  res = [i for n, i in enumerate(flat_list) if i not in flat_list[:n]]
  te.append(res)
print(te[:5])

[['do', 'with', 'your', 'lousy', 'dead', 'bodies', 'what', 'would', 'i', 'want', 'them'], ['alternately', 'described', 'as', 'a', 'symbol', 'of', 'the', 'new', 'grandeur', 'france', 'and'], ['flake'], ['second', 'chance'], ['of', 'dallas', 'it', 'is', 'also', 'symptomatic', 'a', 'change', 'in', 'attitude']]


In [118]:


final=[]
te = list(filter(None, te))
for lst in te:
  result=[]
  for wrd in lst:
    result.append(sample(net, 1, prime = wrd))
  final.append(result)
#print(final)
print(final[:5])


[[0.22788973, 0.1361429, 0.020974463, 0.004824326, 0.013859316, 0.060674068, 0.06666991, 0.21651098, 0.032133516, 0.2990522, 0.060091086], [0.005452682, 0.07409189, 0.09244556, 0.01167673, 0.363444, 0.19453984, 0.007917744, 0.11269789, 0.017419117, 0.050114173, 0.048843816], [0.02425086], [0.0143166315, 0.07910527], [0.19453984, 0.014534784, 0.12576813, 0.045602143, 0.03537219, 0.14426623, 0.01167673, 0.06627879, 0.23338042, 0.19481468]]


In [120]:

perplexity_test=[]
sum=0
cnt=0
c=0
for l in final:
  probability = np.prod(l)
  if probability == 0:
    probability = 0.075
  res = probability ** (-1/len(l))
  perplexity_test.append(res)
  cnt+=1
print(perplexity_test)
avg_test = math.fsum(perplexity_test)/cnt
print("average perplexity of test dataset: "+str(avg_test))

[17.44172110734568, 22.768374624029256, 41.235652053626, 29.71507651007514, 14.242356153528277, 19.23729871830214, 19.552997829391543, 19.83561753613422, 15.381457978318876, 22.755921146348665, 29.732901816461595, 21.19528362165862, 27.959141275940073, 19.814559723146836, 18.496738826629652, 13.05247461557689, 21.622891495996814, 228.11734126709712, 16.302474083820286, 12.46772352197988, 24.86039861235935, 46.0561829572984, 36.17121700288954, 11.079789461072755, 16.872586109981835, 18.096124666729423, 18.149675782550556, 15.213072456365307, 17.289203536054092, 7.968906808996036, 26.983223913255216, 15.073640936845656, 13.283700985943714, 15.370323084627714, 14.422471476034385, 32.849251704436284, 19.221503023725827, 49.74672753226091, 18.495374426279113, 26.584574883232143, 25.466994306834636, 19.328681737213877, 13.224135357738373, 17.88096757459586, 18.04679527762062, 46.6008115408677, 23.687517749455402, 33.691897158723926, 18.147535798858712, 11.672195540620734, 22.087244087398823,

In [127]:
f3 = open("/content/test_file_neural model.txt","w")
k = 0
for ls in te:
  sentence = " ".join(ls)
  f3.write(sentence + "\t" + str(perplexity_test[k]) + "\n")
  k += 1
f3.write("\n")
f3.write("average perplexity of test dataset: "+str(avg_test))



54

In [17]:

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
  
# creating the dataset
data1 = {'Neural Model average perplexity':22.275866192506143, 'Wittenbell Model average perplexity ':2.9793551949239}
data2 = {'Neural Model average perplexity':22.275866192506143, 'KNeserNey Model average perplexity ':5.045214858752456}
data3 = {'Neural Model average train perplexity':22.072964175198386, 'Neural Model average test perplexity':22.275866192506143}
d1 = list(data1.keys())
d2 = list(data1.values())
d3 = list(data2.keys())
d4 = list(data2.values())
d5 = list(data3.keys())
d6 = list(data3.values())
with PdfPages("Visualisation.pdf") as export_pdf:  
    fig = plt.figure(figsize = (10, 5))

    # creating the bar plot
    plt.bar(d1, d2, width = 0.4)
    plt.xlabel("comparision of different models")
    plt.ylabel("Average perplexity")
    plt.title("Model with their respective perplexity")
    export_pdf.savefig()
    plt.close()


    fig = plt.figure(figsize = (10, 5))
    print("\n\n\n\n\n")
    # creating the bar plot
    plt.bar(d3, d4, color ='maroon',
            width = 0.4)

    plt.xlabel("comparision of different models")
    plt.ylabel("Average perplexity")
    plt.title("Model with their respective perplexity")
    export_pdf.savefig()
    plt.close()


    fig = plt.figure(figsize = (10, 5))
    print("\n\n\n\n\n")
    # creating the bar plot
    plt.bar(d5, d6, color ='darkcyan',
            width = 0.4)

    plt.xlabel("comparision of different models")
    plt.ylabel("Average perplexity")
    plt.title("Model with their respective perplexity")
    export_pdf.savefig()
    plt.close()


